In [2]:
! pip install --upgrade numpy

  Using cached numpy-1.24.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.24.2 which is incompatible.
scipy 1.7.0 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.2 which is incompatible.
intel-tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.24.2 which is incompatible.


In [3]:
! pip install --upgrade gensim

  Using cached numpy-1.22.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:
      Successfully uninstalled numpy-1.24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.22.4 which is incompatible.
intel-tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.22.4 which is incompatible.


In [4]:
import gensim
import numpy

In [5]:
numpy.__version__

'1.22.4'

In [6]:
gensim.__version__

'4.3.1'

In [7]:
import pandas as pd
import datetime
from datetime import timedelta

import nltk,string

import re

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import numpy as np
from gensim.models import TfidfModel
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.similarities import SoftCosineSimilarity, Similarity

import pickle
import scipy
from typing import Tuple

from gensim.similarities import SparseTermSimilarityMatrix, SoftCosineSimilarity
import numpy as np
from sentence_transformers import SentenceTransformer, util



In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
# Global Variables to use 
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
stop_words = stopwords.words('english')

HARD_COS_WEIGHT = 0.3
SOFT_COS_WEIGHT = 0.7

KEY_COLUMNS = ['client_property_name', 'workorder_number', 'workorder_desc',
               'workorder_creation_date_ts', 'SLA', 'priority_code', 'type_code']
'''with open("termisim_ups.pkl", "rb") as f:
    termsim_matrix = pickle.load(f)'''

'with open("termisim_ups.pkl", "rb") as f:\n    termsim_matrix = pickle.load(f)'

In [10]:
workorders = pd.read_csv("DATA/SCB_IN_SG.csv")

In [11]:
def preprocess(workorders: pd.DataFrame, splitter: str) -> pd.DataFrame:
    
    date_cols = [cols for cols in workorders.columns if "_ts" in cols]
    workorders[date_cols] = workorders[date_cols].apply(pd.to_datetime)
        
    mask = workorders['workorder_desc'].str.contains(splitter)
    workorders.loc[mask, 'workorder_desc'] = workorders.loc[mask, 'workorder_desc'].str.split(splitter, n=1).str[1].str.lower()
    
    return workorders[KEY_COLUMNS]           

In [12]:
descriptions = preprocess(workorders, ":>")

In [13]:
# Load pre-trained SentenceTransformer model
model = SentenceTransformer('all-distilroberta-v1') #all-MiniLM-L6-v2')

In [14]:
def soft_cosine_similarity(v1, v2, similarity_matrix):
    """Calculate Soft Cosine Similarity between two vectors"""
    return np.dot(v1, similarity_matrix).dot(v2.T)

In [41]:
def get_similar_descriptions(df, location, date, description, window=5):
    """Get similar descriptions for given location, date, and description"""
    
    n_similar=5
    
    # Subset dataframe by location and date window
    start_time = pd.to_datetime(date) - pd.DateOffset(hours=window)
    end_time = pd.to_datetime(date) + pd.DateOffset(hours=window)
    df_subset = df[(df['client_property_name'] == location) & (df['workorder_creation_date_ts'].between(start_time, end_time))]
    
    # Convert descriptions to list
    embeddings = model.encode(df_subset['workorder_desc'].tolist())
    description_embedding = model.encode(description)
    
    # compute soft cosine similarity matrix
    sim_matrix = util.pytorch_cos_sim(description_embedding, embeddings)

    return sim_matrix.numpy()[0], df_subset['workorder_desc'].tolist()

In [45]:
grouped = descriptions.groupby(['client_property_name', pd.Grouper(key='workorder_creation_date_ts', freq='5H')])
# Iterate over groups and get similar descriptions
group_num = 0
more_than_5 = 0
total_number_dupes = 0
for (location, date), group in grouped:
    group_num+=1
    if group.shape[0]>=5:
        more_than_5+=1
        description = group['workorder_desc'].iloc[0]
        sim_mat, work_orders = get_similar_descriptions(descriptions, location, date, description)
        sorted_similarities = (-sim_mat).argsort()[:5]
        if len([cx for cx in sorted_similarities if sim_mat[cx] > 0.9999])>1:
            print("Num dupes:", len([cx for cx in sorted_similarities if sim_mat[cx] > 0.9999])-1)
            print(f"Location: {location}, Date: {date}")
            print(f"Description: {description}")
            print("Duplicate Descriptions:")
            for pos in sorted_similarities:
                if sim_mat[pos] > 0.9999:
                    total_number_dupes+=1
                    print("Work order:", work_orders[pos])
                    print("Similarity Score:", sim_mat[pos])
            print("\n")

Num dupes: 4
Location: Changi Business Park (L3) Zone 3D, Date: 2022-04-18 07:00:00
Description: Hi,

I've forgot the combination code for the locker L3-D-011. Please help in opening it.
Duplicate Descriptions:
Work order: Hi,

I've forgot the combination code for the locker L3-D-011. Please help in opening it.
Similarity Score: 0.9999999
Work order: Hi,

I've forgot the combination code for the locker L3-D-011. Please help in opening it.
Similarity Score: 0.9999999
Work order: Hi,

I've forgot the combination code for the locker L3-D-011. Please help in opening it.
Similarity Score: 0.9999999
Work order: Hi,

I've forgot the combination code for the locker L3-D-011. Please help in opening it.
Similarity Score: 0.9999999
Work order: Hi,

I've forgot the combination code for the locker L3-D-011. Please help in opening it.
Similarity Score: 0.9999999


Num dupes: 4
Location: Changi Business Park 2 - 2nd Floor - Zone C, Date: 2022-06-28 03:00:00
Description: @C10 Level 2 monitor arm is lo

In [46]:
total_number_dupes

21